##**1. 주제**
###**Korean welfare and income**

##**2. 데이터 수집**

https://www.kaggle.com/datasets/hongsean/korea-income-and-welfare


##**3. feature**

1. id
2. year : study conducted
3. wave : from wave 1st in 2005 to wave 14th in 2018

3. region:<br>
(1) Seoul<br>
(2) Kyeong-gi<br>
(3) Kyoung-nam<br>
(4) Kyoung-buk <br>
(5) Chung-nam<br>
(6) Gang-won &. Chung-buk <br>
(7) Jeolla & Jeju

4. income: yearly income in M KRW

5. family_member: no. of family members
6. gender:<br>
 (1) male <br> (2) female

6. year_born

7. education_level: <br>
(1) no education(under 7 yrs-old) <br>(2) no education(7 & over 7 yrs-old) <br>
(3) elementary<br> (4) middle school<br> (5) high school<br> (6) college<br>
(7) university degree<br> (8) MA<br> (9) doctoral degree<br>

8. marriage: marital status<br>
(1) not applicable (under 18)<br> (2) married<br> (3) separated by death <br>
(4) separated<br> (5) not married yet <br> (6) others<br>

9. religion: <br>
(1) have religion<br> (2) do not have

10. occupation
11. company_size

12. reason_none_worker: <br>
(1) no capable<br> (2) in military service<br> (3) studying in school<br>
(4) prepare for school<br> (5) prepare to apply job<br>
(6) house worker<br> (7) caring kids at home <br>(8) nursing <br>
(9) giving-up economic activities <br>(10) no intention to work <br>(11) others

라이브러리 및 데이터불러오기

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
data = pd.read_csv('drive/MyDrive/TF/Korea Income and Welfare.csv')
data

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker
0,10101,2005,1,1,614.0,1,2,1936,2,2,2,,,8
1,10101,2011,7,1,896.0,1,2,1936,2,2,2,,,10
2,10101,2012,8,1,1310.0,1,2,1936,2,2,2,,,10
3,10101,2013,9,1,2208.0,1,2,1936,2,2,2,,,1
4,10101,2014,10,1,864.0,1,2,1936,2,2,2,,,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92852,98000701,2014,10,5,11600.0,6,1,1967,5,1,1,874,1,
92853,98000701,2015,11,5,8327.0,6,1,1967,5,1,1,874,1,
92854,98000701,2016,12,5,7931.0,6,1,1967,5,1,1,874,1,
92855,98000701,2017,13,5,8802.0,5,1,1967,5,1,1,874,1,


사용하지 않을 열 제거

In [8]:
data = data.drop(['id','occupation','company_size','reason_none_worker','wave'],axis=1)

In [9]:
# 데이터 차원 확인
data.shape

(92857, 9)

# 가장 최근 년도 검색

In [10]:
year_max = max(data['year'])
year_max

2018

In [11]:
# 가장 최근년도(2018년) 데이터만 수집
data = data[data['year']==year_max]

In [12]:
len(data) # 2018년 데이터 개수

6331

In [13]:
# 더이상 분석하지 않을 열(year) 제거
data = data.drop(['year'], axis=1)
data

,region,income,family_member,gender,year_born,education_level,marriage,religion
21,1,2330.0,1,2,1945,4,2,1
35,1,815.0,1,1,1948,3,2,2
49,1,2116.0,1,1,1942,7,3,1
64,1,5008.0,5,1,1962,6,1,1
88,2,1332.0,1,2,1940,3,2,1
...,...,...,...,...,...,...,...,...
92817,6,557.0,1,2,1947,2,2,2
92825,6,1948.0,2,1,1933,3,1,2
92833,6,866.0,1,2,1934,3,2,1
92841,6,4118.0,2,1,1956,5,1,1


In [14]:
# 중복된 행 삭제
data.drop_duplicates()
len(data) # 기존 2018년 데이터 행도 6331개 이므로 중복된 행이 없는 것을 확인

6331

# 데이터 분석

In [15]:
# 누락 데이터 찾기
# 누락된 데이터도 없는 것으로 확인
data.isnull().sum()

region             0
income             0
family_member      0
gender             0
year_born          0
education_level    0
marriage           0
religion           0
dtype: int64

In [16]:
data.columns

Index(['region', 'income', 'family_member', 'gender', 'year_born',
       'education_level', 'marriage', 'religion'],
      dtype='object')

In [17]:
#각 column 별로 unique.function 이용해 고유 값 찾은 뒤 오름차순 정렬

print(sorted(data['region'].unique()))
print(sorted(data['family_member'].unique()))
print(sorted(data['gender'].unique()))
print(sorted(data['education_level'].unique()))
print(sorted(data['marriage'].unique()))
print(sorted(data['religion'].unique()))

[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 2]
[2, 3, 4, 5, 6, 7, 8, 9]
[1, 2, 3, 4, 5, 6]
[1, 2]


In [18]:
# 각 column별로 value_counts() function을 이용해 고유한 값들의 개수 출력

print(data['region'].value_counts())
print(data['family_member'].value_counts())
print(data['gender'].value_counts())
print(data['education_level'].value_counts())
print(data['marriage'].value_counts())
print(data['religion'].value_counts())

2    1304
3    1116
7    1108
1     859
4     823
5     602
6     519
Name: region, dtype: int64
1    2122
2    1969
3     990
4     933
5     258
6      50
7       7
8       1
9       1
Name: family_member, dtype: int64
1    4160
2    2171
Name: gender, dtype: int64
5    1725
3    1437
7     996
4     863
2     642
6     473
8     167
9      28
Name: education_level, dtype: int64
1    3556
2    1674
3     580
5     452
4      67
6       2
Name: marriage, dtype: int64
2    3300
1    3031
Name: religion, dtype: int64


In [19]:
# 사용하는 속성중 의미있는 속성의 통계값 확인 
f= pd.DataFrame(data.describe())
f.drop(['religion','region','gender','marriage','region'],axis=1)

,income,family_member,year_born,education_level
count,6331.000000,6331.000000,6331.000000,6331.000000
mean,4273.601279,2.277365,1955.559627,4.591692
std,4509.175617,1.240378,16.390786,1.674147
min,-46897.000000,1.000000,1922.000000,2.000000
25%,1418.500000,1.000000,1942.000000,3.000000
50%,2987.000000,2.000000,1954.000000,5.000000
75%,5984.500000,3.000000,1969.000000,6.000000
max,170204.000000,9.000000,1999.000000,9.000000


In [20]:
# 표준 정규 분포로 표준화
normalization_data = (data - data.mean(axis=0))/data.std(axis=0)
normalization_data

,region,income,family_member,gender,year_born,education_level,marriage,religion
21,-1.364797,-0.431033,-1.029819,1.384148,-0.644242,-0.353429,0.205630,-1.043349
35,-1.364797,-0.767014,-1.029819,-0.722352,-0.461212,-0.950748,0.205630,0.958300
49,-1.364797,-0.478491,-1.029819,-0.722352,-0.827271,1.438529,1.086445,-1.043349
64,-1.364797,0.162868,2.195004,-0.722352,0.392926,0.841209,-0.675185,-1.043349
88,-0.875415,-0.652359,-1.029819,1.384148,-0.949291,-0.950748,0.205630,-1.043349
...,...,...,...,...,...,...,...,...
92817,1.082113,-0.824231,-1.029819,1.384148,-0.522222,-1.548067,0.205630,0.958300
92825,1.082113,-0.515749,-0.223614,-0.722352,-1.376360,-0.950748,-0.675185,0.958300
92833,1.082113,-0.755704,-1.029819,1.384148,-1.315350,-0.950748,0.205630,-1.043349
92841,1.082113,-0.034508,-0.223614,-0.722352,0.026867,0.243890,-0.675185,-1.043349


In [21]:
# 값 치환 (숫자데이터를 문자로 치환해 정보를 도출하기 위함)

data['region'] = data['region'].replace([1,2,3,4,5,6,7],['Seoul','Kyeong-gi','Kyoung-nam','Kyoung-buk','Chung-nam','Gang-won &. Chung-buk', 'Jeolla & Jeju'])
data['marriage'] = data['marriage'].replace([1,2,3,4,5,6],['single','married','single','single','single','single'])
data['education_level'] = data['education_level'].replace([1,2,3,4,5,6,7,8,9],['no education','no education','elementary','middle school','high school','college','university degree','MA','doctoral degree'])
data['gender'] = data['gender'].replace([1,2],['male','female'])
data['religion'] = data['religion'].replace([1,2],['have','do_not'])


In [22]:
#변경된 데이터 확인
data

,region,income,family_member,gender,year_born,education_level,marriage,religion
21,Seoul,2330.0,1,female,1945,middle school,married,have
35,Seoul,815.0,1,male,1948,elementary,married,do_not
49,Seoul,2116.0,1,male,1942,university degree,single,have
64,Seoul,5008.0,5,male,1962,college,single,have
88,Kyeong-gi,1332.0,1,female,1940,elementary,married,have
...,...,...,...,...,...,...,...,...
92817,Gang-won &. Chung-buk,557.0,1,female,1947,no education,married,do_not
92825,Gang-won &. Chung-buk,1948.0,2,male,1933,elementary,single,do_not
92833,Gang-won &. Chung-buk,866.0,1,female,1934,elementary,married,have
92841,Gang-won &. Chung-buk,4118.0,2,male,1956,high school,single,have


In [23]:
# 통계값 확인하기 (수로 표시된 column만 출력)
data.describe()

,income,family_member,year_born
count,6331.000000,6331.000000,6331.000000
mean,4273.601279,2.277365,1955.559627
std,4509.175617,1.240378,16.390786
min,-46897.000000,1.000000,1922.000000
25%,1418.500000,1.000000,1942.000000
50%,2987.000000,2.000000,1954.000000
75%,5984.500000,3.000000,1969.000000
max,170204.000000,9.000000,1999.000000


In [24]:
# 가장 소득이 높은곳 확인
maxIncomeIndex = data[data['income'] == max(data['income'])].index
data[data['income'] == max(data['income'])]

,region,income,family_member,gender,year_born,education_level,marriage,religion
33273,Chung-nam,170204.0,4,male,1969,university degree,single,have


In [25]:
maxIncomeIndex 

Int64Index([33273], dtype='int64')

In [26]:
# 가장 소득이 높은 row의 값
data.loc[maxIncomeIndex]

,region,income,family_member,gender,year_born,education_level,marriage,religion
33273,Chung-nam,170204.0,4,male,1969,university degree,single,have


가장 소득이 높은 사람은 충남지역에 거주하고 있고, </br>종교를 가지고 있으며 4명의 가족 구성원을 가진 1969년생 대졸자 single 남성이다.

In [27]:
# 지역별 평균 소득 확인
data.groupby(['region'])['income'].mean()

region
Chung-nam                4817.839701
Gang-won &. Chung-buk    3754.438921
Jeolla & Jeju            3553.840433
Kyeong-gi                5095.418098
Kyoung-buk               3587.831835
Kyoung-nam               4022.111470
Seoul                    4870.470314
Name: income, dtype: float64

평균 소득이 가장 높은 지역 : 경기도

In [28]:
# 성별 평균 소득 확인
data.groupby(['gender'])['income'].mean()

gender
female    2089.102810
male      5413.636418
Name: income, dtype: float64

평균 소득이 높은 성별: 남성

In [29]:
# 교육 수준과 수입의 관계 
data.groupby(['education_level'])['income'].mean()

education_level
MA                    7736.568862
college               6011.862579
doctoral degree      10287.285714
elementary            2077.236326
high school           5078.830377
middle school         3070.929316
no education          1328.126791
university degree     7413.320582
Name: income, dtype: float64

평균 소득이 가장 높은 교육 수준: 박사(doctoral degree)

In [30]:
# 결혼 상태 별 평균 소득 확인
data.groupby(['marriage'])['income'].mean()

marriage
married    1681.365472
single     5205.403457
Name: income, dtype: float64

single일때 평균 소득 수준이 더 높다

In [31]:
# 가족수와 수입의 관계 
data.groupby(['family_member'])['income'].mean()

family_member
1     1693.207116
2     3461.465414
3     6341.885859
4     8209.908896
5     8274.201550
6     9685.796000
7     8653.142857
8    20106.000000
9     9535.000000
Name: income, dtype: float64

가족 수가 8명일 때 비약적으로 평균 수익이 증가 </br>
-> 극단적인 값에 의한 평균 수익일 가능성이 높음</br>
-> 소득 상위 10%와 하위 10%의 값을 처리해 준 후 다시 구함

In [32]:
# 소득 기준으로 내림차순 정렬
sorted_by_income_data = data.sort_values(by=['income'],ascending = False)
print(sorted_by_income_data)

           region    income  family_member gender  year_born  \
33273   Chung-nam  170204.0              4   male       1969   
32266   Chung-nam   66531.0              2   male       1955   
27612   Kyeong-gi   66100.0              4   male       1976   
69145  Kyoung-buk   45461.0              5   male       1965   
79745   Kyeong-gi   34153.0              2   male       1958   
...           ...       ...            ...    ...        ...   
57036   Chung-nam    -321.0              2   male       1954   
25919   Chung-nam    -699.0              2   male       1954   
56609   Chung-nam   -1727.0              4   male       1973   
45810   Kyeong-gi  -10035.0              2   male       1944   
7935        Seoul  -46897.0              5   male       1974   

         education_level marriage religion  
33273  university degree   single     have  
32266  university degree   single   do_not  
27612  university degree   single     have  
69145        high school   single   do_not  
79745 

In [33]:
# 소득 상위 10% 행
print(sorted_by_income_data.head(int(len(sorted_by_income_data) * 0.1)))

# 소득 하위 10% 행
print(sorted_by_income_data.tail(int(len(sorted_by_income_data) * 0.1)))

              region    income  family_member gender  year_born  \
33273      Chung-nam  170204.0              4   male       1969   
32266      Chung-nam   66531.0              2   male       1955   
27612      Kyeong-gi   66100.0              4   male       1976   
69145     Kyoung-buk   45461.0              5   male       1965   
79745      Kyeong-gi   34153.0              2   male       1958   
...              ...       ...            ...    ...        ...   
16218     Kyoung-nam    9194.0              4   male       1965   
57836  Jeolla & Jeju    9191.0              5   male       1941   
66628  Jeolla & Jeju    9188.0              3   male       1947   
55628      Chung-nam    9182.0              3   male       1956   
35532     Kyoung-nam    9181.0              4   male       1980   

         education_level marriage religion  
33273  university degree   single     have  
32266  university degree   single   do_not  
27612  university degree   single     have  
69145        hi

In [34]:
# 정확한 평균 소득 확인을 위해 상하위 10% 행 삭제(위아래 각각 633개의 데이터 행 삭제)
sorted_by_income_data = sorted_by_income_data.iloc[int(len(sorted_by_income_data) * 0.1):-int(len(sorted_by_income_data) * 0.1)]
print(sorted_by_income_data)

              region  income  family_member  gender  year_born  \
88217          Seoul  9181.0              5  female       1966   
23484     Kyoung-buk  9160.0              3    male       1960   
34000      Chung-nam  9159.0              4    male       1971   
30280  Jeolla & Jeju  9156.0              3    male       1978   
56193      Chung-nam  9154.0              4    male       1971   
...              ...     ...            ...     ...        ...   
68091     Kyoung-buk   879.0              1  female       1935   
79440  Jeolla & Jeju   879.0              1  female       1932   
71405     Kyoung-buk   878.0              1  female       1939   
6633           Seoul   877.0              1  female       1972   
89379          Seoul   877.0              1  female       1938   

         education_level marriage religion  
88217        high school   single     have  
23484        high school   single     have  
34000  university degree   single   do_not  
30280  university degree   

In [35]:
# 소득 상하위 10%제거한 데이터에서 지역별 평균 소득 확인
sorted_by_income_data.groupby(['region'])['income'].mean()

region
Chung-nam                3742.573729
Gang-won &. Chung-buk    3413.257466
Jeolla & Jeju            3447.348951
Kyeong-gi                4103.192424
Kyoung-buk               3113.960889
Kyoung-nam               3475.360046
Seoul                    3915.884985
Name: income, dtype: float64

평균 소득이 가장 높은 지역 : 경기도

In [36]:
# 소득 상하위 10%제거한 데이터에서 성별 평균 소득 확인
sorted_by_income_data.groupby(['gender'])['income'].mean()

gender
female    2346.108566
male      4251.796607
Name: income, dtype: float64

평균 소득이 더 높은 성별 : 남성

In [37]:
# 소득 상하위 10%제거한 데이터에서 교육 수준과 수입의 관계 
sorted_by_income_data.groupby(['education_level'])['income'].mean()

education_level
MA                   5633.478632
college              5176.605528
doctoral degree      5565.235294
elementary           2252.828963
high school          4221.606148
middle school        3036.384955
no education         1674.249034
university degree    5217.430404
Name: income, dtype: float64

평균 소득이 가장 높은 교육 수준 : MA(석사)

In [38]:
# 소득 상하위 10% 제거한 데이터에서 교육 수준과 수입의 관계 
sorted_by_income_data.groupby(['marriage'])['income'].mean()

marriage
married    1959.293626
single     4156.536324
Name: income, dtype: float64

평균 소득이 가장 높은 결혼 상태 : single

---



In [39]:
# 소득 상하위 10% 제거한 데이터에서 가족수와 수입의 관계 
sorted_by_income_data.groupby(['family_member'])['income'].mean()

family_member
1    1976.825131
2    3157.883531
3    5157.515385
4    6112.416794
5    6210.356725
6    6516.960000
7    6383.250000
Name: income, dtype: float64

평균 소득이 가장 높은 가족수 : 6명

# one hot encoding

In [40]:
# one hot encoding
dummy1 = pd.get_dummies(data[['religion']])
dummy3 = pd.get_dummies(data[['gender']])
dummy4 = pd.get_dummies(data[['marriage']])
data['education_level'] = data['education_level'].replace(['no education','no education','elementary','middle school','high school','college','university degree','MA','doctoral degree'],[1,2,3,4,5,6,7,8,9])
data = pd.concat([data, dummy1, dummy3,dummy4], axis=1)
data =data.drop(['religion','region','gender','marriage','region'],axis=1)

In [41]:
data

,income,family_member,year_born,education_level,religion_do_not,religion_have,gender_female,gender_male,marriage_married,marriage_single
21,2330.0,1,1945,4,0,1,1,0,1,0
35,815.0,1,1948,3,1,0,0,1,1,0
49,2116.0,1,1942,7,0,1,0,1,0,1
64,5008.0,5,1962,6,0,1,0,1,0,1
88,1332.0,1,1940,3,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...
92817,557.0,1,1947,2,1,0,1,0,1,0
92825,1948.0,2,1933,3,1,0,0,1,0,1
92833,866.0,1,1934,3,0,1,1,0,1,0
92841,4118.0,2,1956,5,0,1,0,1,0,1


In [42]:
# 표준 졍규 분포로 표준화하기
normalization_data = (data - data.mean(axis=0))/data.std(axis=0)
normalization_data

,income,family_member,year_born,education_level,religion_do_not,religion_have,gender_female,gender_male,marriage_married,marriage_single
21,-0.431033,-1.029819,-0.644242,-0.353429,-1.043349,1.043349,1.384148,-1.384148,1.667789,-1.667789
35,-0.767014,-1.029819,-0.461212,-0.950748,0.958300,-0.958300,-0.722352,0.722352,1.667789,-1.667789
49,-0.478491,-1.029819,-0.827271,1.438529,-1.043349,1.043349,-0.722352,0.722352,-0.599502,0.599502
64,0.162868,2.195004,0.392926,0.841209,-1.043349,1.043349,-0.722352,0.722352,-0.599502,0.599502
88,-0.652359,-1.029819,-0.949291,-0.950748,-1.043349,1.043349,1.384148,-1.384148,1.667789,-1.667789
...,...,...,...,...,...,...,...,...,...,...
92817,-0.824231,-1.029819,-0.522222,-1.548067,0.958300,-0.958300,1.384148,-1.384148,1.667789,-1.667789
92825,-0.515749,-0.223614,-1.376360,-0.950748,0.958300,-0.958300,-0.722352,0.722352,-0.599502,0.599502
92833,-0.755704,-1.029819,-1.315350,-0.950748,-1.043349,1.043349,1.384148,-1.384148,1.667789,-1.667789
92841,-0.034508,-0.223614,0.026867,0.243890,-1.043349,1.043349,-0.722352,0.722352,-0.599502,0.599502


In [43]:
# 첫번째 사람과 거리가 가까운 샘플 번호 
# 정규화를 한 이유 income의 영향이 너무 커서 줄이기 위해 

distance_vector1_1 = np.linalg.norm(normalization_data.iloc[0] - normalization_data.iloc[1:], axis=1, ord=1)
num1_1 = np.argmin(distance_vector1_1) #정규화 한 후 데이터 거리

distance_vector1_2 = np.linalg.norm(normalization_data.iloc[0] - normalization_data.iloc[1:], axis=1, ord=2)
num1_2 = np.argmin(distance_vector1_2) #정규화 한 후 데이터 거리

distance_vector1_inf = np.linalg.norm(normalization_data.iloc[0] - normalization_data.iloc[1:], axis=1, ord=np.inf)
num1_inf = np.argmin(distance_vector1_inf) #정규화 한 후 데이터 거리

distance_vector2 = np.linalg.norm(data.iloc[0] - data.iloc[1:], axis=1, ord=1)
num2 = np.argmin(distance_vector2) # 정규화하기 전 데이터 거리

print('1-norm 거리측정 :', distance_vector1_1,'가장 가까운 데이터 인덱스 : ',num1_1+1)
print('2-norm 거리측정 :', distance_vector1_2,'가장 가까운 데이터 인덱스 : ',num1_2+1)
print('inf-norm 거리측정 :', distance_vector1_inf,'가장 가까운 데이터 인덱스 : ',num1_inf+1)
print('(정규화 하지 않았을때) 1-norm 거리측정 :', distance_vector2,'가장 가까운 데이터 인덱스 : ',num2+1)

1-norm 거리측정 : [ 9.33262946 10.77002646 14.79810995 ...  1.59309919 11.21873908
 15.69770406] 가장 가까운 데이터 인덱스 :  475
2-norm 거리측정 : [4.17025897 4.73315527 5.69306361 ... 0.95529501 4.55744142 6.20668662] 가장 가까운 데이터 인덱스 :  475
inf-norm 거리측정 : [2.10649986 2.26729049 3.22482281 ... 0.67110877 2.26729049 4.03102852] 가장 가까운 데이터 인덱스 :  475
(정규화 하지 않았을때) 1-norm 거리측정 : [1523.  224. 2705. ... 1476. 1805. 4449.] 가장 가까운 데이터 인덱스 :  2837


In [44]:
# 정규화 전
data.iloc[0] , data.iloc[num2+1]

(income              2330.0
 family_member          1.0
 year_born           1945.0
 education_level        4.0
 religion_do_not        0.0
 religion_have          1.0
 gender_female          1.0
 gender_male            0.0
 marriage_married       1.0
 marriage_single        0.0
 Name: 21, dtype: float64,
 income              2330.0
 family_member          1.0
 year_born           1945.0
 education_level        3.0
 religion_do_not        1.0
 religion_have          0.0
 gender_female          1.0
 gender_male            0.0
 marriage_married       1.0
 marriage_single        0.0
 Name: 47491, dtype: float64)

정규화하기 전거리 데이터와 정규화 한 후의 거리 데이터의 수치가 다르다.</br>
이는 다양한 데이터의 수치의 scale의 차이 때문임을 알 수 있다. 

In [45]:
# 정규화 후
data.iloc[0] , data.iloc[num1_1+1]

(income              2330.0
 family_member          1.0
 year_born           1945.0
 education_level        4.0
 religion_do_not        0.0
 religion_have          1.0
 gender_female          1.0
 gender_male            0.0
 marriage_married       1.0
 marriage_single        0.0
 Name: 21, dtype: float64,
 income              2053.0
 family_member          1.0
 year_born           1944.0
 education_level        4.0
 religion_do_not        0.0
 religion_have          1.0
 gender_female          1.0
 gender_male            0.0
 marriage_married       1.0
 marriage_single        0.0
 Name: 8061, dtype: float64)

In [46]:
# 정규화한 코사인
cosine_similarity(np.array(normalization_data.iloc[0]).reshape(1,-1), np.array(normalization_data.iloc[num1_1+1]).reshape(1,-1))

array([[0.99973428]])

In [47]:
normalization_data.iloc[num1_1+1]

income             -0.492463
family_member      -1.029819
year_born          -0.705252
education_level    -0.353429
religion_do_not    -1.043349
religion_have       1.043349
gender_female       1.384148
gender_male        -1.384148
marriage_married    1.667789
marriage_single    -1.667789
Name: 8061, dtype: float64

In [48]:
data.iloc[num1_1+1]

income              2053.0
family_member          1.0
year_born           1944.0
education_level        4.0
religion_do_not        0.0
religion_have          1.0
gender_female          1.0
gender_male            0.0
marriage_married       1.0
marriage_single        0.0
Name: 8061, dtype: float64

In [49]:
#상관계수 구하기
normalization_data.corr().style.background_gradient(cmap = 'coolwarm')

,income,family_member,year_born,education_level,religion_do_not,religion_have,gender_female,gender_male,marriage_married,marriage_single
income,1.000000,0.546047,0.415581,0.471028,0.042024,-0.042024,-0.350003,0.350003,-0.344696,0.344696
family_member,0.546047,1.000000,0.452891,0.447023,0.072579,-0.072579,-0.513012,0.513012,-0.492150,0.492150
year_born,0.415581,0.452891,1.000000,0.659478,0.186195,-0.186195,-0.296624,0.296624,-0.503677,0.503677
education_level,0.471028,0.447023,0.659478,1.000000,0.075821,-0.075821,-0.434835,0.434835,-0.521712,0.521712
religion_do_not,0.042024,0.072579,0.186195,0.075821,1.000000,-1.000000,-0.145626,0.145626,-0.126588,0.126588
religion_have,-0.042024,-0.072579,-0.186195,-0.075821,-1.000000,1.000000,0.145626,-0.145626,0.126588,-0.126588
gender_female,-0.350003,-0.513012,-0.296624,-0.434835,-0.145626,0.145626,1.000000,-1.000000,0.683555,-0.683555
gender_male,0.350003,0.513012,0.296624,0.434835,0.145626,-0.145626,-1.000000,1.000000,-0.683555,0.683555
marriage_married,-0.344696,-0.492150,-0.503677,-0.521712,-0.126588,0.126588,0.683555,-0.683555,1.000000,-1.000000
marriage_single,0.344696,0.492150,0.503677,0.521712,0.126588,-0.126588,-0.683555,0.683555,-1.000000,1.000000


양의 상관관계를 가질수록 붉은 색을 띈다.

In [50]:
# 공분산 구하기
normalization_data.cov().style.background_gradient(cmap = 'coolwarm')

,income,family_member,year_born,education_level,religion_do_not,religion_have,gender_female,gender_male,marriage_married,marriage_single
income,1.000000,0.546047,0.415581,0.471028,0.042024,-0.042024,-0.350003,0.350003,-0.344696,0.344696
family_member,0.546047,1.000000,0.452891,0.447023,0.072579,-0.072579,-0.513012,0.513012,-0.492150,0.492150
year_born,0.415581,0.452891,1.000000,0.659478,0.186195,-0.186195,-0.296624,0.296624,-0.503677,0.503677
education_level,0.471028,0.447023,0.659478,1.000000,0.075821,-0.075821,-0.434835,0.434835,-0.521712,0.521712
religion_do_not,0.042024,0.072579,0.186195,0.075821,1.000000,-1.000000,-0.145626,0.145626,-0.126588,0.126588
religion_have,-0.042024,-0.072579,-0.186195,-0.075821,-1.000000,1.000000,0.145626,-0.145626,0.126588,-0.126588
gender_female,-0.350003,-0.513012,-0.296624,-0.434835,-0.145626,0.145626,1.000000,-1.000000,0.683555,-0.683555
gender_male,0.350003,0.513012,0.296624,0.434835,0.145626,-0.145626,-1.000000,1.000000,-0.683555,0.683555
marriage_married,-0.344696,-0.492150,-0.503677,-0.521712,-0.126588,0.126588,0.683555,-0.683555,1.000000,-1.000000
marriage_single,0.344696,0.492150,0.503677,0.521712,0.126588,-0.126588,-0.683555,0.683555,-1.000000,1.000000


양의 상관관계를 가질수록 붉은 색을 띈다.

# 데이터 분석 과정 및 결과

##0. 데이터 정보
- 지역 </br>
Kyeong-gi : 1304 </br>
Kyoung-nam : 1116</br>
Jeolla & Jeju : 1108</br>
Seoul : 859</br>
Kyoung-buk : 823</br>
Chung-nam : 602</br>
Gang-won &. Chung-buk : 519</br>

- 가족수</br>
1 :   2122</br>
2 :   1969</br>
3 :   990</br>
4 :    933</br>
5 :    258</br>
6 :    50</br>
7  :     7</br>
8  :     1</br>
9  :     1</br>

- 결혼</br>
18세 이하:    3556</br>
결혼:     1674</br>
사별:     580</br>
이혼:     452</br>
미혼      67</br>
기타:       2</br>


- 종교</br>
종교있음 :   3300</br>
종교없음  :  3031</br>

- 성별</br>
male :   4160</br>
female  :  2171</br>

- 교육 수준</br>
high school :   1725</br>
elementary  :  1437</br>
university degree  :   996</br>
middle school  :  863</br>
no education(7 & over 7 yrs-old) :   642</br>
college  :   473</br>
MA  :   167</br>
doctoral degree :  28</br>


##1. 데이터 전처리
- 가장 최근 연도인 2018년 데이터만 추출 </br>
- 분석에서 사용하지 않을 열('id','occupation', 'company_size','reason_none_worker','wave', 'year) 제거 </br>
- 중복된 행 삭제 및 누락 데이터 확인</br>
- 각 column별 분포 및 통계값 확인</br>
- *92857행 14열 --> 6331행 8열*

##2. 데이터 분석
####[데이터 표준화]</br>
- 각 column별 분포 및 통계값 확인</br>
- 동일한 척도에서 비교하기위해 표준 정규분포표로 표준화 진행</br>
- 표준화 진행 후 상관계수 출력</br>

####[전체 데이터] </br>
- 가장 소득이 높은 사람 -> 충남/종교o/4명가족/1969년생/- 대졸자/single/남성</br>
- 지역별 평균 소득확인 -> 평균 소득이 가장 높은 지역: 경기도</br>
- 성별 평균 소득확인 -> 평균 소득이 높은 성별: 남성</br>
- 교육 수준별 평균 소득 확인 -> 평균 소득이 높은 교육 수준: 박사(doctoral degree)</br>
- 결혼 상태별 평균 소득 확인 -> 평균 소득이 높은 결혼 상태: single</br>
- 가족 수별 평균 소득 확인 -> 평균 소득이 높은 가족 수: 8명</br>

####[소득 상위10%, 하위 10%를 제거한 데이터] </br>
- 지역별 평균 소득확인 -> 평균 소득이 가장 높은 지역: 경기도</br>
- 성별 평균 소득확인 -> 평균 소득이 높은 성별: 남성</br>
- 교육 수준별 평균 소득 확인 -> 평균 소득이 높은 교육 수준: 석사(MA)</br>
- 결혼 상태별 평균 소득 확인 -> 평균 소득이 높은 결혼 상태: single</br>
- 가족 수별 평균 소득 확인 -> 평균 소득이 높은 가족 수: 6명</br>

####[소득이외에 데이터를 통해 알 수 있는 점] </br> 
- family_member와 income은 양의 상관관계(0.5460) -> 가족 수가 많을수록 수입이 높음

- year_born과 education level은 양의 상관관계 (0.6594)
	->나이가 어릴 수록 교육수준이 높음
- year_born과 marriage_married은 음의 상관관계 (-0.5217)
	-> 나이가 어릴수록 배우자가 없을 확률 높음
- gender_female과 family member은 양의 상관관계(0.513012) -> 여성일 수록 가족수가 많음.
- gender_male과 family member은 양의 상관관계(-0.513012) -> 남성일 수록 가족수가 적음.
- gender_female과 marriage_married은 양의 상관관계 (0.683555) ->  여성일수록 배우자가 있음.
- gender_male과 marriage_married은 음의 상관관계 (-0.683555) -> 남성일수록 배우자가 없음.

# 결론


1. 전체 데이터와 소득 상위10%, 하위 10% 데이터를 제거한 데이터를 비교해보면 평균 소득이 가장 높은 교육 수준이 박사(doctoral degree)에서 MA(석사)로 바뀌고 평균 소득이 가장 높은 가족 구성원 수가 8명에서 6명으로 바뀌었다. 이는 이상치의 영향을 받는다는 걸 알 수 있다.

2. 가장 소득이 높은 사람이 1969년생
4명의 가족 구성원을 가진 single 남성으로 나타났는데, </br>
이는 single의 범위에 사별, 이혼 등의 사례까지 포함되어있어 다음과 같이 나옴을 알 수 있다.

3. 정규화 하지 않은 데이터는 scale의 차이로 인해 소득의 영향을 많이 받아 거리와 상관계수를 측정할 때 정규화하기 전과 후의 차이가 생기는 것을 알 수 있다.